In [1]:
import scrapy
import numpy as np
from scrapy.crawler import CrawlerProcess
import re
import json

In [2]:
class Diputados(scrapy.Spider):
    name = "spider_diputados"
    
    def start_requests(self):
        links = [urlCurr + i for i in terminaciones]
        for link in links:
            yield scrapy.Request( url = link, headers={'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:66.0) Gecko/20100101 Firefox/66.0'}, 
                              callback = self.parse )
    
    def parse(self,response):
        nombre = response.css('strong::text').extract_first()
        nombre = limpiaNom(nombre) # cambié esto
        idDipu = ''.join([c for c in nombre[1:] if c.isupper()])
        cumple = response.css('strong font::text').extract()[5]
        curp = creaCurp(nombre,cumple)
        dipus[curp]={}
        dipus.get(curp,{})['CURP']=curp # Agregué esto para 
        dipus.get(curp,{})['Nombre']=nombre # cambié aquí
        dipus.get(curp,{})['TipoElección']=response.css('strong font::text').extract()[0]
        dipus.get(curp,{})['Entidad']=response.css('strong font::text').extract()[1]
        parti=response.xpath('//tr/td/table//p//img/@src').extract()[0]
        dipus.get(curp,{})['Partido']=obtienePartido(parti)
        distri1 =response.css('strong font::text').extract()[2]
        distri2 =response.css('strong font::text').extract()[3]  # PARTE RARA
        dipus.get(curp,{})['Distrito'] = distri1 +""+ distri2
        dipus.get(curp,{})['Correo']=response.css('strong font::text').extract()[4]
        dipus.get(curp,{})['Onomástico']=cumple
        supl=response.css('strong font::text').extract()[6]
        dipus.get(curp,{})['Suplente']=limpiaText(supl)
        dipus.get(curp,{})['Comisiones']=response.xpath('//a[contains(@href, "integrantes_de_comision")]/text()').getall()
        dipus.get(curp,{})['Último grado de estudios']=response.xpath('//td[@class="textoNegro"]/text()').extract_first()
        link_asis=urlDipu + response.css('a.linkBlancoSin::attr(href)').extract()[2][1:-1]
        link_votaciones=urlDipu + response.css('a.linkBlancoSin::attr(href)').extract()[3][1:]
        #yield response.follow(url=link_asis, headers={'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:66.0) Gecko/20100101 Firefox/66.0'},meta={'curp':curp},
        #                     callback=self.parse2)
        yield response.follow(url=link_votaciones, headers={'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:66.0) Gecko/20100101 Firefox/66.0'},meta={'curp':curp},
                              callback=self.parse4)
    
    def parse2(self,response):
        curp = response.meta['curp']
        referencias = response.xpath('//a[@class="linkVerde"]/@href').extract()
        links = [urlDipu + referencia for referencia in referencias]
        for li in links:
            yield response.follow(url=li, headers={'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:66.0) Gecko/20100101 Firefox/66.0'},meta={'curp':curp},
                              callback=self.parse3)
    
    def parse3(self,response):
        curp = response.meta['curp']
        prueba = response.xpath('//tr/td/div/font/text()').extract()
        dias = list(filter(lambda a: (a == 'A' or a =='AC'), prueba))
        dipus.get(curp,{})['Asistencias']=dipus.get(curp,{}).get('Asistencias',0) + len(dias)

    def parse4(self,response):
        curp = response.meta['curp']
        referencias = response.xpath('//a[@class="linkVerde"]/@href').extract()
        links = [urlDipu + referencia for referencia in referencias]
        for li in links:
            yield response.follow(url=li, headers={'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:66.0) Gecko/20100101 Firefox/66.0'},meta={'curp':curp},
                              callback=self.parse5)
                
    def parse5(self,response):
        curp = response.meta['curp']
        aux = response.xpath('//td[@class="smallVerde"]/text()').extract()
        aux2={}
        for i in range(1,len(aux),4):
            votos = {}
            votacion = response.xpath('//td[@class="smallVerde"]/text()').extract()[i]
            posi = response.xpath('//td[@class="smallVerde"]/text()').extract()[i+2]
            if(votacion in por_votaciones.keys()):
                por_votaciones[votacion][curp]=posi
            else:
                    por_votaciones[votacion] = {"ID_voto":limpiaVotacion(votacion),"Nombre_Votación":votacion, curp:posi}
            votos['votación']=votacion
            votos['posición']=posi
            aux2['votos']=aux2.get('votos', []) + [votos]
            #dipus.get(curp,{})['votos']=dipus.get(curp,{}).get('votos', []) + [votos]
        dipus.get(curp,{})['votos']=dipus.get(curp,{}).get('votos',0)+len(aux2.get('votos', [])) # Se comenta si la de arriba se descomenta


        

In [3]:
def creaCurp(nombre,cumple):
    
    '''
    iniciales = ''.join([c for c in nombre[1:] if c.isupper()])
    c = cumple.replace('-', ' ').split(' ')
    if(len(c[0])==1):
        c[0]="0"+c[0] 
    curp = iniciales[0:3]+c[0]+c[1][0:3]+c[2][-2:]
    '''
    curp = nombre.replace(' ', '')
    curp = curp.upper()
    curp = curp.replace('Á', 'A')
    curp = curp.replace('É', 'E')
    curp = curp.replace('Í', 'I')
    curp = curp.replace('Ó', 'O')
    curp = curp.replace('Ú', 'U')
    curp = curp.replace('Ñ', 'N')
    return curp

In [4]:
def limpiaText(nom):
    resp = re.sub('\s+',' ',nom)
    resp = resp[0:len(resp)-1]
    return resp

In [5]:
def limpiaNom(nom):
    resp = nom.replace('Dip. ', '')
    resp = resp.replace(' (LICENCIA)', '')
    return resp

In [6]:
def limpiaVotacion(nombreVot):
    nombreVot = nombreVot.replace('(', '')
    nombreVot = nombreVot.replace(')', '')
    nombreVot = nombreVot.replace(' SE ', ' ')
    nombreVot = nombreVot.replace('DECRETO', '')
    nombreVot = nombreVot.replace(' QUE ', ' ')
    nombreVot = nombreVot.replace(' SE ', ' ')
    nombreVot = nombreVot.replace(' DE ', ' ')
    nombreVot = nombreVot.replace(' A ', ' ')
    nombreVot = nombreVot.replace(' LA ', ' ')
    nombreVot = nombreVot.replace(' EL ', ' ')
    nombreVot = nombreVot.replace(' POR ', ' ')
    nombreVot = nombreVot.replace(' PARA ', ' ')
    nombreVot = nombreVot.replace(' LOS ', ' ')
    nombreVot = nombreVot.replace(' LAS ', ' ')
    nombreVot = nombreVot.replace(' Y ', ' ')
    nombreVot = nombreVot.replace(' E ', ' ')
    nombreVot = nombreVot.replace(' CON ', ' ')
    nombreVot = nombreVot.replace(' LEY ', ' ')
    nombreVot = nombreVot.replace(' EN ', ' ')
    nombreVot = nombreVot.replace(' LO ', ' ')
    nombreVot = nombreVot.replace(' DEL ', ' ')
    nombreVot = nombreVot.replace('.', ' ')
    nombreVot = nombreVot.replace(';', ' ')
    nombreVot = nombreVot.replace(',', ' ')
    nombreVot = nombreVot.replace('  ', ' ')
    nombreFin=""
    for s in nombreVot.split():
        nombreFin+=s[0]
    return nombreFin
    

In [7]:
def obtienePartido(imagen):
    res=""
    if(imagen=='images/pan.png'):
        res = "PAN"
    elif(imagen=='images/Logo_PT.png'):
        res = "PT"
    elif(imagen=='images/LogoMorena.jpg'):
        res = "Morena"
    elif(imagen=='images/encuentro.png'):
        res = "Encuentro Social"
    elif(imagen=='images/logo_movimiento_ciudadano.png'):
        res = "Movimiento Ciudadano"
    elif(imagen=='images/pri01.png'):
        res = "PRI"
    elif(imagen=='images/logvrd.jpg'):
        res = "Partido Verde"
    elif(imagen=='images/prd01.png'):
        res = "PRD"
    elif(imagen=='images/logo_SP.jpg'):
        res = "Sin Partido"
    elif(imagen=="images/panal.gif"):
        res = "Nueva Alianza"
    elif(imagen=="images/independiente.png"):
        res = "Independiente"
    else:
         res = imagen
    return res
                                    

In [8]:
dipus={}
curp = "h"
urlDipu = "http://sitllxiii.diputados.gob.mx"
urlCurr = "http://sitllxiii.diputados.gob.mx/curricula.php?dipt="

por_votaciones = {}

terminaciones = []
for i in range(0,5):
    for j in range(0,10):
        for k in range(0,10):
            terminaciones.append(str(i)+str(j)+str(k))
terminaciones.append(str(500))
terminaciones.pop(0)


# Run the Spider
process = CrawlerProcess()
process.crawl(Diputados)
process.start()


2021-04-08 23:56:06 [scrapy.utils.log] INFO: Scrapy 2.4.1 started (bot: scrapybot)
2021-04-08 23:56:06 [scrapy.utils.log] INFO: Versions: lxml 4.6.2.0, libxml2 2.9.5, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 21.2.0, Python 3.8.2 (tags/v3.8.2:7b3ab59, Feb 25 2020, 23:03:10) [MSC v.1916 64 bit (AMD64)], pyOpenSSL 20.0.1 (OpenSSL 1.1.1j  16 Feb 2021), cryptography 3.4.6, Platform Windows-10-10.0.19041-SP0
2021-04-08 23:56:06 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2021-04-08 23:56:06 [scrapy.crawler] INFO: Overridden settings:
{}
2021-04-08 23:56:06 [scrapy.extensions.telnet] INFO: Telnet Password: b9626889b788ec51
2021-04-08 23:56:06 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.logstats.LogStats']
2021-04-08 23:56:07 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddl

2021-04-08 23:56:08 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitllxiii.diputados.gob.mx/votaciones_diputados_xperiodonplxiii.php?dipt=009> (referer: http://sitllxiii.diputados.gob.mx/curricula.php?dipt=009)
2021-04-08 23:56:08 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitllxiii.diputados.gob.mx/votaciones_diputados_xperiodonplxiii.php?dipt=014> (referer: http://sitllxiii.diputados.gob.mx/curricula.php?dipt=014)
2021-04-08 23:56:08 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitllxiii.diputados.gob.mx/votaciones_diputados_xperiodonplxiii.php?dipt=011> (referer: http://sitllxiii.diputados.gob.mx/curricula.php?dipt=011)
2021-04-08 23:56:08 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitllxiii.diputados.gob.mx/votaciones_diputados_xperiodonplxiii.php?dipt=015> (referer: http://sitllxiii.diputados.gob.mx/curricula.php?dipt=015)
2021-04-08 23:56:08 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitllxiii.diputados.gob.mx/votaciones_diputado

2021-04-08 23:56:27 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitllxiii.diputados.gob.mx./votaciones_por_pernplxiii.php?iddipt=21&pert=5> (referer: http://sitllxiii.diputados.gob.mx/votaciones_diputados_xperiodonplxiii.php?dipt=021)
2021-04-08 23:56:28 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitllxiii.diputados.gob.mx./votaciones_por_pernplxiii.php?iddipt=21&pert=1> (referer: http://sitllxiii.diputados.gob.mx/votaciones_diputados_xperiodonplxiii.php?dipt=021)
2021-04-08 23:56:28 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitllxiii.diputados.gob.mx./votaciones_por_pernplxiii.php?iddipt=24&pert=6> (referer: http://sitllxiii.diputados.gob.mx/votaciones_diputados_xperiodonplxiii.php?dipt=024)
2021-04-08 23:56:28 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitllxiii.diputados.gob.mx./votaciones_por_pernplxiii.php?iddipt=24&pert=13> (referer: http://sitllxiii.diputados.gob.mx/votaciones_diputados_xperiodonplxiii.php?dipt=024)
2021-04-08 23:56:28

2021-04-08 23:56:54 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitllxiii.diputados.gob.mx./votaciones_por_pernplxiii.php?iddipt=25&pert=13> (referer: http://sitllxiii.diputados.gob.mx/votaciones_diputados_xperiodonplxiii.php?dipt=025)
2021-04-08 23:56:54 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitllxiii.diputados.gob.mx./votaciones_por_pernplxiii.php?iddipt=25&pert=5> (referer: http://sitllxiii.diputados.gob.mx/votaciones_diputados_xperiodonplxiii.php?dipt=025)
2021-04-08 23:56:54 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitllxiii.diputados.gob.mx./votaciones_por_pernplxiii.php?iddipt=19&pert=3> (referer: http://sitllxiii.diputados.gob.mx/votaciones_diputados_xperiodonplxiii.php?dipt=019)
2021-04-08 23:56:54 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitllxiii.diputados.gob.mx./votaciones_por_pernplxiii.php?iddipt=19&pert=1> (referer: http://sitllxiii.diputados.gob.mx/votaciones_diputados_xperiodonplxiii.php?dipt=019)
2021-04-08 23:56:59

2021-04-08 23:57:14 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitllxiii.diputados.gob.mx./votaciones_por_pernplxiii.php?iddipt=11&pert=9> (referer: http://sitllxiii.diputados.gob.mx/votaciones_diputados_xperiodonplxiii.php?dipt=011)
2021-04-08 23:57:14 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitllxiii.diputados.gob.mx./votaciones_por_pernplxiii.php?iddipt=11&pert=1> (referer: http://sitllxiii.diputados.gob.mx/votaciones_diputados_xperiodonplxiii.php?dipt=011)
2021-04-08 23:57:14 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitllxiii.diputados.gob.mx./votaciones_por_pernplxiii.php?iddipt=11&pert=5> (referer: http://sitllxiii.diputados.gob.mx/votaciones_diputados_xperiodonplxiii.php?dipt=011)
2021-04-08 23:57:14 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitllxiii.diputados.gob.mx./votaciones_por_pernplxiii.php?iddipt=11&pert=7> (referer: http://sitllxiii.diputados.gob.mx/votaciones_diputados_xperiodonplxiii.php?dipt=011)
2021-04-08 23:57:19 

...

2021-04-09 00:41:48 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitllxiii.diputados.gob.mx./votaciones_por_pernplxiii.php?iddipt=481&pert=9> (referer: http://sitllxiii.diputados.gob.mx/votaciones_diputados_xperiodonplxiii.php?dipt=481)
2021-04-09 00:41:48 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitllxiii.diputados.gob.mx./votaciones_por_pernplxiii.php?iddipt=481&pert=7> (referer: http://sitllxiii.diputados.gob.mx/votaciones_diputados_xperiodonplxiii.php?dipt=481)
2021-04-09 00:41:48 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitllxiii.diputados.gob.mx./votaciones_por_pernplxiii.php?iddipt=482&pert=13> (referer: http://sitllxiii.diputados.gob.mx/votaciones_diputados_xperiodonplxiii.php?dipt=482)
2021-04-09 00:41:49 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitllxiii.diputados.gob.mx./votaciones_por_pernplxiii.php?iddipt=482&pert=6> (referer: http://sitllxiii.diputados.gob.mx/votaciones_diputados_xperiodonplxiii.php?dipt=482)
2021-04-09 00:4

2021-04-09 00:42:08 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitllxiii.diputados.gob.mx/curricula.php?dipt=488> (referer: None)
2021-04-09 00:42:08 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitllxiii.diputados.gob.mx./votaciones_por_pernplxiii.php?iddipt=478&pert=1> (referer: http://sitllxiii.diputados.gob.mx/votaciones_diputados_xperiodonplxiii.php?dipt=478)
2021-04-09 00:42:11 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitllxiii.diputados.gob.mx./votaciones_por_pernplxiii.php?iddipt=479&pert=11> (referer: http://sitllxiii.diputados.gob.mx/votaciones_diputados_xperiodonplxiii.php?dipt=479)
2021-04-09 00:42:11 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitllxiii.diputados.gob.mx./votaciones_por_pernplxiii.php?iddipt=479&pert=13> (referer: http://sitllxiii.diputados.gob.mx/votaciones_diputados_xperiodonplxiii.php?dipt=479)
2021-04-09 00:42:11 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitllxiii.diputados.gob.mx./votaciones_por_pe

2021-04-09 00:42:20 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitllxiii.diputados.gob.mx./votaciones_por_pernplxiii.php?iddipt=491&pert=13> (referer: http://sitllxiii.diputados.gob.mx/votaciones_diputados_xperiodonplxiii.php?dipt=491)
2021-04-09 00:42:20 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitllxiii.diputados.gob.mx./votaciones_por_pernplxiii.php?iddipt=489&pert=13> (referer: http://sitllxiii.diputados.gob.mx/votaciones_diputados_xperiodonplxiii.php?dipt=489)
2021-04-09 00:42:20 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitllxiii.diputados.gob.mx./votaciones_por_pernplxiii.php?iddipt=488&pert=1> (referer: http://sitllxiii.diputados.gob.mx/votaciones_diputados_xperiodonplxiii.php?dipt=488)
2021-04-09 00:42:25 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitllxiii.diputados.gob.mx./votaciones_por_pernplxiii.php?iddipt=492&pert=13> (referer: http://sitllxiii.diputados.gob.mx/votaciones_diputados_xperiodonplxiii.php?dipt=492)
2021-04-09 00

2021-04-09 00:42:51 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitllxiii.diputados.gob.mx./votaciones_por_pernplxiii.php?iddipt=500&pert=3> (referer: http://sitllxiii.diputados.gob.mx/votaciones_diputados_xperiodonplxiii.php?dipt=500)
2021-04-09 00:42:51 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitllxiii.diputados.gob.mx./votaciones_por_pernplxiii.php?iddipt=500&pert=5> (referer: http://sitllxiii.diputados.gob.mx/votaciones_diputados_xperiodonplxiii.php?dipt=500)
2021-04-09 00:42:51 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitllxiii.diputados.gob.mx./votaciones_por_pernplxiii.php?iddipt=500&pert=6> (referer: http://sitllxiii.diputados.gob.mx/votaciones_diputados_xperiodonplxiii.php?dipt=500)
2021-04-09 00:42:51 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitllxiii.diputados.gob.mx./votaciones_por_pernplxiii.php?iddipt=500&pert=9> (referer: http://sitllxiii.diputados.gob.mx/votaciones_diputados_xperiodonplxiii.php?dipt=500)
2021-04-09 00:42

2021-04-09 00:43:13 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitllxiii.diputados.gob.mx./votaciones_por_pernplxiii.php?iddipt=490&pert=6> (referer: http://sitllxiii.diputados.gob.mx/votaciones_diputados_xperiodonplxiii.php?dipt=490)
2021-04-09 00:43:13 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitllxiii.diputados.gob.mx./votaciones_por_pernplxiii.php?iddipt=490&pert=5> (referer: http://sitllxiii.diputados.gob.mx/votaciones_diputados_xperiodonplxiii.php?dipt=490)
2021-04-09 00:43:13 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitllxiii.diputados.gob.mx./votaciones_por_pernplxiii.php?iddipt=490&pert=3> (referer: http://sitllxiii.diputados.gob.mx/votaciones_diputados_xperiodonplxiii.php?dipt=490)
2021-04-09 00:43:13 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://sitllxiii.diputados.gob.mx./votaciones_por_pernplxiii.php?iddipt=490&pert=1> (referer: http://sitllxiii.diputados.gob.mx/votaciones_diputados_xperiodonplxiii.php?dipt=490)
2021-04-09 00:43

In [9]:
list_of_dipus = [value for value in dipus.values()]


[{'CURP': 'ARLETTEIVETTEMUNOZCERVANTES',
  'Nombre': 'Arlette Ivette Muñoz Cervantes',
  'TipoElección': 'Mayoría Relativa ',
  'Entidad': 'Aguascalientes ',
  'Partido': 'PAN',
  'Distrito': '2 G-203 ',
  'Correo': 'arlette.munoz@congreso.gob.mx ',
  'Onomástico': '16-septiembre ',
  'Suplente': 'Cynthia Susana Zaragoza Galván',
  'Comisiones': [' Comunicaciones  '],
  'Último grado de estudios': 'Licenciatura',
  'votos': 714},
 {'CURP': 'LUZARGELIAPANIAGUAFIGUEROA',
  'Nombre': 'Luz Argelia Paniagua Figueroa',
  'TipoElección': 'Mayoría Relativa ',
  'Entidad': 'Baja California ',
  'Partido': 'PAN',
  'Distrito': '2 H-243 ',
  'Correo': 'luz.paniagua@congreso.gob.mx ',
  'Onomástico': '28-febrero ',
  'Suplente': 'Margarita Mercado Medina',
  'Comisiones': [' Ciencia y Tecnología  '],
  'Último grado de estudios': 'Licenciatura',
  'votos': 710},
 {'CURP': 'GERARDOFEDERICOSALASDIAZ',
  'Nombre': 'Gerardo Federico Salas Díaz',
  'TipoElección': 'Mayoría Relativa ',
  'Entidad': 'Agu

In [10]:
list_of_votos = [value for value in por_votaciones.values()]


In [11]:
for key, value in dipus.items():
    print('\n',key, ' : \n\n', value)


 ARLETTEIVETTEMUNOZCERVANTES  : 

 {'CURP': 'ARLETTEIVETTEMUNOZCERVANTES', 'Nombre': 'Arlette Ivette Muñoz Cervantes', 'TipoElección': 'Mayoría Relativa ', 'Entidad': 'Aguascalientes ', 'Partido': 'PAN', 'Distrito': '2 G-203 ', 'Correo': 'arlette.munoz@congreso.gob.mx ', 'Onomástico': '16-septiembre ', 'Suplente': 'Cynthia Susana Zaragoza Galván', 'Comisiones': [' Comunicaciones  '], 'Último grado de estudios': 'Licenciatura', 'votos': 714}

 GERARDOFEDERICOSALASDIAZ  : 

 {'CURP': 'GERARDOFEDERICOSALASDIAZ', 'Nombre': 'Gerardo Federico Salas Díaz', 'TipoElección': 'Mayoría Relativa ', 'Entidad': 'Aguascalientes ', 'Partido': 'PAN', 'Distrito': '1 K-498 ', 'Correo': 'gerardo.salas@congreso.gob.mx ', 'Onomástico': '25-mayo ', 'Suplente': 'José de Jesús Valdez Gómez', 'Comisiones': [' Agricultura y Sistemas de Riego  '], 'Último grado de estudios': 'Maestría', 'votos': 661}

 JORGERAMOSHERNANDEZ  : 

 {'CURP': 'JORGERAMOSHERNANDEZ', 'Nombre': 'Jorge Ramos Hernández', 'TipoElección': 'Ma

In [12]:
for key, value in por_votaciones.items():
    print('\n',key,': ')
    for curp, posi in value.items():
        print('\n',curp, ': ', posi)


 DECRETO POR EL QUE SE EXPIDE LA LEY GENERAL DEL SISTEMA NACIONAL ANTICORRUPCIÓN, LA LEY GENERAL DE RESPONSABILIDADES ADMINISTRATIVAS, Y LA LEY ORGÁNICA DEL TRIBUNAL FEDERAL DE JUSTICIA ADMINISTRATIVA, SOBRE LAS OBSERVACIONES DEL EJECUTIVO FEDERAL (EN LO GENERAL Y EN LO PARTICULAR LOS ARTÍCULOS NO RESERVADOS)  : 

 ID_voto :  EGSNAGRAOTFJASOEFGPANR

 Nombre_Votación :  DECRETO POR EL QUE SE EXPIDE LA LEY GENERAL DEL SISTEMA NACIONAL ANTICORRUPCIÓN, LA LEY GENERAL DE RESPONSABILIDADES ADMINISTRATIVAS, Y LA LEY ORGÁNICA DEL TRIBUNAL FEDERAL DE JUSTICIA ADMINISTRATIVA, SOBRE LAS OBSERVACIONES DEL EJECUTIVO FEDERAL (EN LO GENERAL Y EN LO PARTICULAR LOS ARTÍCULOS NO RESERVADOS) 

 GERARDOFEDERICOSALASDIAZ :  A favor

 DECRETO POR EL QUE SE EXPIDE LA LEY GENERAL DEL SISTEMA NACIONAL ANTICORRUPCIÓN, LA LEY GENERAL DE RESPONSABILIDADES ADMINISTRATIVAS, Y LA LEY ORGÁNICA DEL TRIBUNAL FEDERAL DE JUSTICIA ADMINISTRATIVA, SOBRE LAS OBSERVACIONES DEL EJECUTIVO FEDERAL (EN LO PARTICULAR LOS ARTÍCUL


 ID_voto :  RPSA3FTGP

 Nombre_Votación :  DECRETO POR EL QUE SE REFORMA EL PARRAFO SEGUNDO DEL ARTICULO 3o. DE LA LEY FEDERAL DEL TRABAJO (EN LO GENERAL Y EN LO PARTICULAR) 

 GERARDOFEDERICOSALASDIAZ :  A favor

 DECRETO POR EL QUE SE REFORMAN, ADICIONAN Y DEREOGAN DIVERSAS DISPOSICIONES DE LA LEY FEDERAL DE PROCEDIMIENTO ADMINISTRATIVO (EN LO GENERAL Y EN LO PARTICULAR)  : 

 ID_voto :  RADDDFPAGP

 Nombre_Votación :  DECRETO POR EL QUE SE REFORMAN, ADICIONAN Y DEREOGAN DIVERSAS DISPOSICIONES DE LA LEY FEDERAL DE PROCEDIMIENTO ADMINISTRATIVO (EN LO GENERAL Y EN LO PARTICULAR) 

 GERARDOFEDERICOSALASDIAZ :  A favor

 DECRETO POR EL QUE SE ADICIONA UNA FRACCION XII AL ARTICULO 6o. DE LA LEY GENERAL DE SALUD (EN LO GENERAL Y EN LO PARTICULAR)  : 

 ID_voto :  AUFXAA6GSGP

 Nombre_Votación :  DECRETO POR EL QUE SE ADICIONA UNA FRACCION XII AL ARTICULO 6o. DE LA LEY GENERAL DE SALUD (EN LO GENERAL Y EN LO PARTICULAR) 

 GERARDOFEDERICOSALASDIAZ :  A favor

 DECRETO POR EL QUE SE REFORMA

 Nombre_Votación :  DECRETO POR EL QUE SE EXPIDE EL REGLAMENTO DE TRANSPARENCIA, ACCESO A LA INFORMACIÓN PÚBLICA Y PROTECCIÓN DE DATOS PERSONALES DE LA CÁMARA DE DIPUTADOS DEL CONGRESO DE LA UNIÓN (EN LO PARTICULAR LOS ARTÍCULOS RESERVADOS, EN TÉRMINOS DEL DICTAMEN Y CON LAS MODIFICACIONES INCORPORADAS POR LA COMISIÓN) 

 GERARDOFEDERICOSALASDIAZ :  A favor

 DECRETO POR EL QUE SE EXPIDE LA LEY FEDERAL DE DECLARACIÓN ESPECIAL DE AUSENCIA PARA PERSONAS DESAPARECIDAS Y POR EL QUE SE REFORMAN DIVERSAS DISPOSICIONES DE LA LEY FEDERAL DEL TRABAJO; DE LA LEY FEDERAL DE LOS TRABAJADORES AL SERVICIO DEL ESTADO, REGLAMENTARIA DEL APARTADO B) DEL ARTÍCULO 123 CONSTITUCIONAL; DE LA LEY DEL SEGURO SOCIAL; DE LA LEY DEL INSTITUTO DE SEGURIDAD Y SERVICIOS SOCIALES DE LOS TRABAJADORES DEL ESTADO; DE LA LEY GENERAL DE TÍTULOS Y OPERACIONES DE CRÉDITO; DE LA LEY DE INSTITUCIONES DE CRÉDITO Y DE LA LEY AGRARIA (EN LO GENERAL Y EN LO PARTICULAR)  : 

 ID_voto :  EFDEAPDRDDFTFTASERABA1CSSISSSTEGTOCICAGP



 GERARDOFEDERICOSALASDIAZ :  A favor

 DECRETO POR EL QUE SE REFORMAN Y ADICIONAN LOS ARTÍCULOS 12 Y 14 DE LA LEY GENERAL DE EDUCACIÓN (EN LO GENERAL Y EN LO PARTICULAR)  : 

 ID_voto :  RAA11GEGP

 Nombre_Votación :  DECRETO POR EL QUE SE REFORMAN Y ADICIONAN LOS ARTÍCULOS 12 Y 14 DE LA LEY GENERAL DE EDUCACIÓN (EN LO GENERAL Y EN LO PARTICULAR) 

 GERARDOFEDERICOSALASDIAZ :  A favor

 DECRETO POR EL QUE SE REFORMA EL ARTÍCULO 323 TER DEL CÓDIGO CIVIL FEDERAL (EN LO GENERAL Y EN LO PARTICULAR)  : 

 ID_voto :  RA3TCCFGP

 Nombre_Votación :  DECRETO POR EL QUE SE REFORMA EL ARTÍCULO 323 TER DEL CÓDIGO CIVIL FEDERAL (EN LO GENERAL Y EN LO PARTICULAR) 

 GERARDOFEDERICOSALASDIAZ :  A favor

 DECRETO POR EL QUE SE REFORMA EL ARTÍCULO 11 DE LA LEY ORGÁNICA DE LA PROCURADURÍA GENERAL DE LA REPÚBLICA (EN LO GENERAL Y EN LO PARTICULAR)  : 

 ID_voto :  RA1OPGRGP

 Nombre_Votación :  DECRETO POR EL QUE SE REFORMA EL ARTÍCULO 11 DE LA LEY ORGÁNICA DE LA PROCURADURÍA GENERAL DE LA REPÚBLICA (EN 

 Nombre_Votación :  DECRETO POR EL QUE SE EXPIDE LA LEY GENERAL DE ASENTAMIENTOS HUMANOS, ORDENAMIENTO TERRITORIAL Y DESARROLLO URBANO Y SE REFORMA EL ARTICULO 3o. DE LA LEY DE PLANEACION (EN LO PARTICULAR LOS ARTICULOS RESERVADOS EN SUS TERMINOS) 

 GERARDOFEDERICOSALASDIAZ :  A favor

 DECRETO POR EL QUE SE ADICIONAN DIVERSAS DISPOSICIONES DE LA LEY DE LA COMISION NACIONAL PARA EL DESARROLLO DE LOS PUEBLOS INDIGENAS Y DE LA LEY GENERAL DE ACCESO DE LAS MUJERES A UNA VIDA LIBRE DE VIOLENCIA (EN LO GENERAL Y EN LO PARTICULAR)  : 

 ID_voto :  ADDCNDPIGAMUVLVGP

 Nombre_Votación :  DECRETO POR EL QUE SE ADICIONAN DIVERSAS DISPOSICIONES DE LA LEY DE LA COMISION NACIONAL PARA EL DESARROLLO DE LOS PUEBLOS INDIGENAS Y DE LA LEY GENERAL DE ACCESO DE LAS MUJERES A UNA VIDA LIBRE DE VIOLENCIA (EN LO GENERAL Y EN LO PARTICULAR) 

 GERARDOFEDERICOSALASDIAZ :  A favor

 DECRETO POR EL QUE SE ADICIONAN UN TERCER, CUARTO, QUINTO, SEXTO, OCTAVO, NOVENO Y DECIMO PARRAFOS AL ARTICULO 51 BIS 1 DE LA LE

 GERARDOFEDERICOSALASDIAZ :  Ausente

 DECRETO POR EL QUE EL HONORABLE CONGRESO DE LA UNION DECLARA EL 5 DE MAYO DE CADA AÑO COMO DIA NACIONAL DE LA LUCHA CONTRA LA HIPERTENSION PULMONAR (EN LO GENERAL Y EN LO PARTICULAR)  : 

 ID_voto :  EHCUD5MCACDNLCHPGP

 Nombre_Votación :  DECRETO POR EL QUE EL HONORABLE CONGRESO DE LA UNION DECLARA EL 5 DE MAYO DE CADA AÑO COMO DIA NACIONAL DE LA LUCHA CONTRA LA HIPERTENSION PULMONAR (EN LO GENERAL Y EN LO PARTICULAR) 

 GERARDOFEDERICOSALASDIAZ :  Ausente

 DECRETO POR EL QUE SE REFORMA EL ARTICULO 137 DEL CODIGO NACIONAL DE PROCEDIMIENTOS PENALES(EN LO GENERAL Y EN LO PARTICULAR)  : 

 ID_voto :  RA1CNPPGP

 Nombre_Votación :  DECRETO POR EL QUE SE REFORMA EL ARTICULO 137 DEL CODIGO NACIONAL DE PROCEDIMIENTOS PENALES(EN LO GENERAL Y EN LO PARTICULAR) 

 GERARDOFEDERICOSALASDIAZ :  Ausente

 DECRETO POR EL QUE SE REFORMA EL ARTICULO 167 DEL CODIGO NACIONAL DE PROCEDIMIENTOS PENALES(EN LO GENERAL Y EN LO PARTICULAR)  : 

 ID_voto :  RA1CNPPGP

 N

 ID_voto :  RA89IabcdTPA11FIIIA11A1PPSA1111FPSDCMHGP

 Nombre_Votación :  DECRETO POR EL QUE SE REFORMAN LOS ARTÍCULOS 8; 9 INCISOS a), b), c), d) Y TERCER PÁRRAFO; ARTÍCULOS 11 Y 12, FRACCIONES I, II Y III; ARTÍCULOS 13, 14; ARTÍCULO 15, PÁRRAFOS PRIMERO Y SEGUNDO; ARTÍCULOS 16, 17, 18 Y 19 DE LA LEY FEDERAL PARA PREVENIR Y SANCIONAR LOS DELITOS COMETIDOS EN MATERIA DE HIDROCARBUROS (EN LO GENERAL Y EN LO PARTICULAR) 

 GERARDOFEDERICOSALASDIAZ :  Ausente

 DECRETO POR EL QUE EXPIDE LA LEY GENERAL DE CULTURA Y DERECHOS CULTURALES (EN LO GENERAL Y EN LO PARTICULAR LOS ARTÍCULOS NO RESERVADOS)  : 

 ID_voto :  EGCDCGPANR

 Nombre_Votación :  DECRETO POR EL QUE EXPIDE LA LEY GENERAL DE CULTURA Y DERECHOS CULTURALES (EN LO GENERAL Y EN LO PARTICULAR LOS ARTÍCULOS NO RESERVADOS) 

 GERARDOFEDERICOSALASDIAZ :  Ausente

 DECRETO POR EL QUE EXPIDE LA LEY GENERAL DE CULTURA Y DERECHOS CULTURALES (EN LO PARTICULAR LOS ARTÍCULOS RESERVADOS, EN SUS TÉRMINOS)  : 

 ID_voto :  EGCDCPARST

 Nombre_V

In [11]:
with open('diputados_LXIII.json', 'a') as f:
            json.dump(list_of_dipus, f, indent=4, ensure_ascii=False) # antes dipus

In [12]:
with open('votaciones_LXIII.json', 'a') as f:
            json.dump(list_of_votos, f, indent=4, ensure_ascii=False)